In [14]:
import pandas as pd
import numpy as np
import warnings
import tqdm
warnings.filterwarnings("ignore")

In [15]:
TIME_CUT = "1973-01-01"
SPLIT_DATE_TRAIN = "2020-01-01"
MAG_TH = 5
GEO_SPLIT = 1

In [26]:
df = pd.read_csv("../data/usgs_data_small.csv")
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df

,time,longitude,latitude,depth,mag,magType
0,1940-01-06 20:04:35.170000,25.814000,35.373000,15.00,5.84,mw
1,1940-01-06 09:15:39.210000,151.498000,45.077000,25.00,6.07,mw
2,1940-01-05 09:42:55.570000,-116.367333,33.173167,6.00,3.42,ml
3,1940-01-05 07:20:50.460000,-119.442000,32.929333,6.00,3.97,ml
4,1940-01-04 21:44:55.390000,37.926000,40.415000,15.00,5.51,mw
...,...,...,...,...,...,...
4152281,2023-09-24 02:20:39.520000,-117.314167,34.071333,16.11,1.09,ml
4152282,2023-09-24 02:11:22.160000,-155.259000,58.195167,2.55,0.14,ml
4152283,2023-09-24 02:10:46.071000,-119.668800,40.223400,8.40,0.70,ml
4152284,2023-09-24 02:06:35.040000,-123.223333,39.279500,4.43,1.91,md


In [4]:
df["time_disc"] = df["time"].apply(lambda x: x[:10])
df = df[df["time"] >= TIME_CUT]
df["time"] = pd.to_datetime(df["time"], format="mixed")
df

,time,longitude,latitude,depth,mag,magType,time_disc
52274,1973-01-07 00:12:09.960,-155.432333,19.771500,22.834,2.34,ml,1973-01-07
52275,1973-01-06 23:15:49.900,167.487000,-15.496000,123.000,5.30,mb,1973-01-06
52276,1973-01-06 21:01:50.900,48.256000,33.098000,63.000,4.80,mb,1973-01-06
52277,1973-01-06 17:26:40.100,-157.021500,20.207667,4.900,3.23,ml,1973-01-06
52278,1973-01-06 16:52:41.900,166.384000,-14.665000,36.000,6.10,mb,1973-01-06
...,...,...,...,...,...,...,...
4152281,2023-09-24 02:20:39.520,-117.314167,34.071333,16.110,1.09,ml,2023-09-24
4152282,2023-09-24 02:11:22.160,-155.259000,58.195167,2.550,0.14,ml,2023-09-24
4152283,2023-09-24 02:10:46.071,-119.668800,40.223400,8.400,0.70,ml,2023-09-24
4152284,2023-09-24 02:06:35.040,-123.223333,39.279500,4.430,1.91,md,2023-09-24


In [5]:
df["longitude_disc"] = (df["longitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
df["latitude_disc"] = (df["latitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
df["pos"] = df["latitude_disc"].astype(str) + "_" + df["longitude_disc"].astype(str)

In [6]:
df

,time,longitude,latitude,depth,mag,magType,time_disc,longitude_disc,latitude_disc,pos
52274,1973-01-07 00:12:09.960,-155.432333,19.771500,22.834,2.34,ml,1973-01-07,-156,19,19_-156
52275,1973-01-06 23:15:49.900,167.487000,-15.496000,123.000,5.30,mb,1973-01-06,167,-16,-16_167
52276,1973-01-06 21:01:50.900,48.256000,33.098000,63.000,4.80,mb,1973-01-06,48,33,33_48
52277,1973-01-06 17:26:40.100,-157.021500,20.207667,4.900,3.23,ml,1973-01-06,-158,20,20_-158
52278,1973-01-06 16:52:41.900,166.384000,-14.665000,36.000,6.10,mb,1973-01-06,166,-15,-15_166
...,...,...,...,...,...,...,...,...,...,...
4152281,2023-09-24 02:20:39.520,-117.314167,34.071333,16.110,1.09,ml,2023-09-24,-118,34,34_-118
4152282,2023-09-24 02:11:22.160,-155.259000,58.195167,2.550,0.14,ml,2023-09-24,-156,58,58_-156
4152283,2023-09-24 02:10:46.071,-119.668800,40.223400,8.400,0.70,ml,2023-09-24,-120,40,40_-120
4152284,2023-09-24 02:06:35.040,-123.223333,39.279500,4.430,1.91,md,2023-09-24,-124,39,39_-124


In [16]:
df_tp = pd.read_csv("../data/all.csv")
df_tp.drop_duplicates(inplace=True)
df_tp

,plate,lat,lon
0,am,30.754,132.824
1,am,30.970,132.965
2,am,31.216,133.197
3,am,31.515,133.500
4,am,31.882,134.042
...,...,...,...
12314,yz,20.561,112.784
12315,yz,20.137,113.030
12316,yz,19.713,113.274
12317,yz,19.288,113.517


In [9]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [10]:
def find_min_dist(df_tp, x, y):
    df_tp["dist"] = haversine_distance(x, y, df_tp["lat"], df_tp["lon"])
    min_dist = df_tp["dist"].min()
    plates = df_tp[df_tp["dist"] == min_dist].sort_values("plate")["plate"].tolist()
    plates = "_".join(plates)
    return min_dist, plates

In [11]:
def add_region_info(df, df_tp):
    region2plate = {}
    region2dist = {}
    df["lat_cent"] = df["latitude_disc"] + 0.5
    df["lon_cent"] = df["longitude_disc"] + 0.5
    for pos in tqdm.tqdm(df["pos"].unique()):
        x, y = pos.split("_")
        x, y = float(x), float(y)
        dist, plate = find_min_dist(df_tp, x + 0.5, y + 0.5)
        region2plate[pos] = plate
        region2dist[pos] = dist
    df["plate_region"] = df["pos"].map(region2plate)
    df["dist_region"] = df["pos"].map(region2dist)
    return df

In [12]:
def add_tectonic_info(df, df_tp):
    coordinates = list(zip(df['latitude'], df['longitude']))
    results = list(tqdm.tqdm(map(lambda x: find_min_dist(df_tp, x[0], x[1]), coordinates), total=len(coordinates)))
    df[["dist", "plate"]] = results
    return df

In [13]:
def preprocess_magtype(df):
    n = 17
    magtype2id = {magtype: i+1 for i, magtype in enumerate(df[df["time"] < SPLIT_DATE_TRAIN]["magType"].value_counts().index[:n])}
    magtype2id.update({magtype: n+1 for magtype in df[df["time"] < SPLIT_DATE_TRAIN]["magType"].value_counts().index[n:]})
    df["magType"] = df["magType"].map(magtype2id)
    return df, magtype2id

In [14]:
def add_features(df, df_tp, mag_th):
    df_final = None
    df["time"] = pd.to_datetime(df["time"], format="mixed")
    df["time_disc"] = pd.to_datetime(df["time_disc"], format="mixed")
    df = add_region_info(df, df_tp)
    df = add_tectonic_info(df, df_tp)
    for pos in tqdm.tqdm(df["pos"].unique()):
        dfs = []
        tmp = df[df["pos"] == pos]
        tmp.sort_values("time", inplace=True)
        for time in tmp["time_disc"].unique():
            tmp_t0 = tmp[tmp["time_disc"] == time]
            t1 = time + pd.DateOffset(months=1)
            tmp_t1 = tmp[(tmp["time_disc"] > time) & (tmp["time_disc"] <= t1)]
            if tmp_t1.empty:
                max_mag = -1e8
            else:
                max_mag = tmp_t1["mag"].max()
            tmp_t0["label"] = 0 if max_mag < mag_th else 1
            dfs.append(tmp_t0)
        df_tmp = pd.concat(dfs)
        df_final = pd.concat([df_final, df_tmp])
    return df_final

In [17]:
df_final = add_features(df, df_tp, MAG_TH)

100%|██████████| 13057/13057 [2:28:43<00:00,  1.46it/s]  


In [18]:
df_final

,time,longitude,latitude,depth,mag,magType,time_disc,longitude_disc,latitude_disc,pos,lat_cent,lon_cent,plate_region,dist_region,dist,plate,label
102,1973-01-01 08:58:11.460,-155.360833,19.443667,7.302,1.85,1,1973-01-01,-156,19,19_-156,19.5,-155.5,jf_pa,3527.668174,3522.498687,jf_pa,0
88,1973-01-02 06:14:29.740,-155.253167,19.314833,7.041,1.93,1,1973-01-02,-156,19,19_-156,19.5,-155.5,jf_pa,3527.668174,3525.575831,jf_pa,0
87,1973-01-02 07:19:25.150,-155.204333,19.326000,6.589,2.21,1,1973-01-02,-156,19,19_-156,19.5,-155.5,jf_pa,3527.668174,3521.228924,jf_pa,0
73,1973-01-03 05:59:23.410,-155.281000,19.430167,15.044,1.99,1,1973-01-03,-156,19,19_-156,19.5,-155.5,jf_pa,3527.668174,3517.998774,jf_pa,0
72,1973-01-03 06:30:18.700,-155.277333,19.409500,11.126,1.88,1,1973-01-03,-156,19,19_-156,19.5,-155.5,jf_pa,3527.668174,3519.447349,jf_pa,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097391,2023-09-17 13:56:58.198,-16.551100,36.338800,10.000,4.30,3,2023-09-17,-17,36,36_-17,36.5,-16.5,eu_nu,106.166869,123.068500,eu_nu,0
4097887,2023-09-29 22:17:32.686,-63.741300,60.600500,10.000,4.20,3,2023-09-29,-64,60,60_-64,60.5,-63.5,eu_na,1730.151056,1743.541742,eu_na,0
4098399,2023-09-28 12:50:20.146,-110.487800,32.234100,5.000,3.10,1,2023-09-28,-111,32,32_-111,32.5,-110.5,na_pa,364.340172,356.687073,na_pa,0
4098683,2023-09-27 19:47:21.596,93.112900,0.862200,10.000,5.50,11,2023-09-27,93,0,0_93,0.5,93.5,cp_in,50.773571,35.611755,cp_in,0


In [ ]:
df_final.to_csv("../data/with_features_notmapped.csv", index=False)

In [24]:
def make_mapping(df, col, n, SPLIT_DATE_TRAIN):
    type2id = {magtype: i+1 for i, magtype in enumerate(df[df["time"] < SPLIT_DATE_TRAIN][col].value_counts().index[:n])}
    type2id.update({magtype: n+1 for magtype in df[df["time"] < SPLIT_DATE_TRAIN][col].value_counts().index[n:]})
    type2id.update({x: n+1 for x in df[col].unique() if x not in type2id})
    df[col] = df[col].map(type2id)
    return df, type2id

In [25]:
df_final = pd.read_csv("../data/with_features_notmapped.csv")

In [26]:
df_final, magtype2id = make_mapping(df_final, "magType", 17, SPLIT_DATE_TRAIN)
df_final, plate2id = make_mapping(df_final, "plate", 60, SPLIT_DATE_TRAIN)
df_final, plate_region2id = make_mapping(df_final, "plate_region", 50, SPLIT_DATE_TRAIN)

In [32]:
pd.DataFrame(magtype2id.items(), columns=["magType", "magType_id"]).to_csv("../data/magtype2id.csv", index=False)
pd.DataFrame(plate2id.items(), columns=["plate", "plate_id"]).to_csv("../data/plate2id.csv", index=False)
pd.DataFrame(plate_region2id.items(), columns=["plate_region", "plate_region_id"]).to_csv("../data/plate_region2id.csv", index=False)

In [36]:
df_final.to_csv("../data/with_features.csv", index=False)